<a href="https://colab.research.google.com/github/jaimebaldeon/Financial-Stock-Prediction/blob/morningstar/financialStocks_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prueba investpy
!pip install git+https://github.com/alvarobartt/investpy.git@developer
import investpy


  Cloning https://github.com/alvarobartt/investpy.git (to revision developer) to /tmp/pip-req-build-s6bpcvim
  Running command git clone -q https://github.com/alvarobartt/investpy.git /tmp/pip-req-build-s6bpcvim
  Running command git checkout -b developer --track origin/developer
  Switched to a new branch 'developer'
  Branch 'developer' set up to track remote branch 'developer' from 'origin'.
     |████████████████████████████████| 245kB 5.2MB/s 
     |████████████████████████████████| 5.5MB 16.8MB/s 
     |████████████████████████████████| 512kB 41.4MB/s 
  Created wheel for investpy: filename=investpy-1.0-cp36-none-any.whl size=4466670 sha256=7b6dbb3e7dc01b5710f9f7c8f20f4035349d0bd2fbcc3d20d1f5954f39009cf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ni8nd71/wheels/8b/ac/54/fa4dbb4808e366ad5011a2a09b0a8e86907f0f997dc173c8a9
Successfully built investpy
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Found exist

In [ ]:

df = investpy.get_stock_historical_data(stock='BABA',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2020')
print(df)

              Open    High     Low   Close    Volume Currency
Date                                                         
2014-09-22   92.70   92.95   89.50   89.89  66657828      USD
2014-09-23   88.94   90.48   86.62   87.17  39009788      USD
2014-09-24   88.47   90.57   87.22   90.57  32088108      USD
2014-09-25   91.09   91.50   88.50   88.92  28597506      USD
2014-09-26   89.73   90.46   88.66   90.46  18339716      USD
...            ...     ...     ...     ...       ...      ...
2019-12-24  215.01  215.43  213.35  214.26   5522987      USD
2019-12-26  214.65  216.99  214.33  216.38   6917445      USD
2019-12-27  217.99  218.11  215.17  215.47  11678669      USD
2019-12-30  215.78  215.78  211.76  212.91   8745942      USD
2019-12-31  212.00  213.64  210.73  212.10   6776071      USD

[1329 rows x 6 columns]


In [ ]:
data = investpy.get_crypto_historical_data(crypto='bitcoin', from_date='01/01/2014', to_date='01/01/2019')
print(data)

              Open    High     Low   Close  Volume Currency
Date                                                       
2014-01-01   805.9   829.9   771.0   815.9   10757      USD
2014-01-02   815.9   886.2   810.5   856.9   12812      USD
2014-01-03   856.9   888.2   839.4   884.3    9709      USD
2014-01-04   884.3   932.2   848.3   924.7   14239      USD
2014-01-05   924.7  1029.9   911.4  1014.7   21374      USD
...            ...     ...     ...     ...     ...      ...
2018-12-28  3587.1  3900.3  3565.5  3861.6  565237      USD
2018-12-29  3861.6  3899.6  3696.0  3706.8  505408      USD
2018-12-30  3708.2  3837.7  3682.5  3815.0  519173      USD
2018-12-31  3815.1  3819.6  3658.8  3709.4  545825      USD
2019-01-01  3709.5  3814.3  3664.4  3809.4  469107      USD

[1827 rows x 6 columns]


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur

In [24]:
def convert_to_int(df): 
  col = df.columns
  for col in df: 
    temp = df[col].to_string().replace(',','').split('\n') #     Replace comma with space and clean out extra spaces
    temp.pop(0) #Remove first erronous element
    df.index = pd.to_datetime(df.index) # Convert index to datetime 
    df[col]= [i[10:].strip() for i in temp] # stripping off nth characters
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int) # Transform columns from   object to numeric
  return df
  
def clean_data(is_data):
  Fin_st = pd.DataFrame(is_data[0:])
  Fin_st.columns = Fin_st.iloc[0] # Name columns to first row of dataframe
  Fin_st = Fin_st.iloc[1:,] # start to read 1st row
  Fin_st = Fin_st.T # transpose dataframe
  Fin_st.columns = Fin_st.iloc[0] #Name columns to first row of dataframe
  Fin_st.drop(Fin_st.index[0],inplace=True) #Drop first index row
  Fin_st.index.name = '' # Remove the index name
  Fin_st.rename(index={'ttm': '12/31/2020'},inplace=True) #Rename ttm in index columns to end of the year
  #Fin_st = Fin_st[Fin_st.columns[:-5]] # remove last 5 irrelevant columns
  return Fin_st

def extract_data(url, fin_st):
  read_data = ur.urlopen(url).read()
  soup_is = BeautifulSoup(read_data, 'lxml')
  print(soup_is)
  div = []
  for l in soup_is.find_all('div'):
    # Find all divs in site
    #print(l.string)
    if l.string is not None:
      div.append(str(l.string))
  div = list(filter(None, div))
  #print(div[15:])
  div = div[15:]
  # Add missing values in div
  div[0] = 'Annual'
  if fin_st is 'is':
    div = div[:6] + ['Total Revenue'] + div[6:]
    div = div[:24] + ['Operating Expense'] + div[24:]
    div = div[:36] + ['Net Non Operating Interest Income Expense'] + div[36:]
    div = div[:42] + ['Other Income Expense'] + div[42:]
    div = div[:60] + ['Net Income Common Stockholders'] + div[60:]
  elif fin_st is 'bs':
    div = div[:5] + ['Total Assets'] + div[5:]
    div = div[:10] + ['Total Liabilities Net Minority Interest'] + div[10:]
    div = div[:15] + ['Total Equity Gross Minority Interest'] + div[15:]
  elif fin_st is 'cf':
    div = div[:6] + ['Operating Cash Flow'] + div[6:]
    div = div[:12] + ['Investing Cash Flow'] + div[12:]
    div = div[:18] + ['Financing Cash Flow'] + div[18:]
    div = div[:24] + ['End Cash Position'] + div[24:]
  else:
    return False
  #print(new_ls)
  return div

In [26]:
# Stock symbol
index= 'MOMO'

# URL link 
url_is = 'https://finance.yahoo.com/quote/' + index + '/financials?p=' + index
url_bs = 'https://finance.yahoo.com/quote/' + index +'/balance-sheet?p=' + index
url_cf = 'https://finance.yahoo.com/quote/' + index + '/cash-flow?p='+ index
print(url_is)


https://finance.yahoo.com/quote/MOMO/financials?p=MOMO


In [12]:
##  INCOME STATEMENT

new_ls = extract_data(url_is, 'is')
is_data = list(zip(*[iter(new_ls)]*6))
Income_st = clean_data(is_data)
Income_st = convert_to_int(Income_st)
Income_st

Annual,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,"24,102",Diluted NI Available to Com Stockholders,Basic EPS,Diluted EPS,Basic Average Shares,Diluted Average Shares,Total Operating Income as Reported,Total Expenses,Net Income from Continuing & Discontinued Operation,Normalized Income,Interest Income,Interest Expense,Net Interest Income,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-31,16886254,8495438,8390816,4626470,3764346,365146,-17211,4112281,883041,0,3220479,3220479,0,0,0,0,3764346,13121908,3220479,3233994,444587,79441,365146,4191722,4553181,8495438,361459,3220479,-17211,-17211,4570392,0,-3696
2019-12-31,17015089,8492096,8522993,4968294,3554699,328931,-15711,3867919,883801,-17319,2970890,2970890,0,0,207658,225603,3554699,13460390,2970890,2983011,407542,78611,328931,3946530,0,8492096,356191,2970890,-15711,-15711,4318432,0,-3590
2018-12-31,13408421,7182897,6225524,2959232,3266292,216443,-43200,3439535,699648,-23350,2815775,2815775,0,0,203505,216542,3266292,10142129,2815775,2850188,272946,56503,216443,3496038,0,7182897,241268,2815775,-43200,-43200,3780506,0,-8787
2017-12-31,8606583,4238915,4367667,2011685,2355982,141248,-28635,2468596,430763,48660,2079819,2079819,0,0,197275,207633,2355982,6250600,2079819,2103457,141248,0,141248,2355982,0,4238915,81191,2079819,-28635,-28635,2465808,0,-4997
2016-12-31,3840602,1676671,2163931,1160649,1003282,56897,-40031,1020148,35663,38447,986388,986388,0,0,188668,203521,1003282,2837320,1008587,1047219,56897,0,56897,1003282,0,1676671,58356,1008587,-40031,-40031,1101669,0,-1399


In [27]:
##  BALANCE SHEET

new_ls = extract_data(url_bs, 'bs')
is_data = list(zip(*[iter(new_ls)]*5))
Balance_sh = clean_data(is_data)
Balance_sh = convert_to_int(Balance_sh)
#Balance_sh

<!DOCTYPE html>
<html class="NoJs desktop" id="atomic" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>Momo Inc. (MOMO) Balance Sheet - Yahoo Finance</title><meta content="balance sheet,asset,liabilities,shareholders equity,return on equity,cash equivalents,retained earning" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" property="twitter:dnt"/><meta content="90376669494" property="fb:app_id"/><meta content="#400090" name="theme-color"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Get the annual and quarterly balance sheet of Momo Inc. (MOMO) including details of assets, liabilities and shareholders' equity." lang="en-US" name="description"/><meta content="guce.yahoo.com" name="oath:guce:consent-host"/><meta content="A9862C0E6E1BE95BCE0BF3D0298FD58B" name="msvalidate.01"

In [ ]:
##  CASH FLOW

new_ls = extract_data(url_cf, 'cf')
is_data = list(zip(*[iter(new_ls)]*6))
CashFlow_st = clean_data(is_data)
CashFlow_st = convert_to_int(CashFlow_st)
CashFlow_st

In [21]:
# EBIT
EBIT = Income_st['Operating Income']
# EBITDA
#print(Income_st[['Operating Income', "Reconciled Depreciation"]])
EBITDA = EBIT + Income_st["Reconciled Depreciation"] # Income_st["EBITDA"] 

# FCF (Operating Cash Flow - |Investing Cash Flow (CAPEX)|)
# Find LFCF

# Otherwise, compute FCF
#print(CashFlow_st[['Operating Cash Flow','Investing Cash Flow'] ])
FCF = CashFlow_st['Operating Cash Flow'] + CashFlow_st['Investing Cash Flow'] 

# ENTERPRISE VALUE (Market Cap - Short-Term Debt + L-T Debt - Cash And Cash Equivalents)


# EV/EBIT

# EV/EBITDA

# EV/FCF

# PER

FCF

Annual      Operating Cash Flow  Investing Cash Flow
                                                    
2020-12-31              4816473             -3739444
2019-12-31              5448886             -4029919
2018-12-31              3327718            -10034004
2017-12-31              2791714              -178299
2016-12-31              1515908              -821785



2020-12-31    1077029
2019-12-31    1418967
2018-12-31   -6706286
2017-12-31    2613415
2016-12-31     694123
dtype: int64